In [13]:
import os, sys
from pathlib import Path

In [2]:
# set to dev-head on server
# sys.path.insert(0, '../../../fastai')

In [14]:
import fastai
fastai.__version__

'1.0.52.dev0'

##### Toy Example to create inode sort-discepancy through compression
create three files but in an order not equal to alphabetical, then view the inode for the directory before/after compression

In [16]:
cmd = '''
mkdir demo-dogs/dogs
touch demo-dogs/dogs/dog_1.txt
touch demo-dogs/dogs/dog_3.txt
touch demo-dogs/dogs/dog_2.txt
ls -i demo-dogs/dogs
tar -cf demo-dogs/dogs.tar demo-dogs/dogs --sort=name
mkdir demo-dogs/new-dogs
tar -xf demo-dogs/dogs.tar -C demo-dogs/new-dogs/
ls -i demo-dogs/new-dogs/demo-dogs/dogs
'''

s_cmd = ' && '.join(cmd.split('\n')[1:-1])
# s_cmd

! $s_cmd

4325413 dog_1.txt  4325415 dog_2.txt  4325414 dog_3.txt
4325419 dog_1.txt  4325420 dog_2.txt  4325421 dog_3.txt


From ls printout you can see inode is *not* in alphanumeric order for original direcotry (because the order of creation) but after compression/decompression, the inode *is* in alphanumeric order in the second directory.

In [15]:
# DON'T RUN THIS if proceeding to next example
# remove the created dir's to re-run;
cmd = '''
rm demo-dogs/new-dogs -r
rm demo-dogs/dogs -r
'''
s_cmd = ' && '.join(cmd.split('\n')[1:-1])

! $s_cmd

##### Demonstrate fatai functions load out of order
this notebook is run on both GCP-server and laptop(win10) in both windows env and WSL env: the order of loading is preserved before/after compression but varies between machines.

In [17]:
from fastai.data_block import _get_files

def is_local(name='DESKTOP-5VTC260'):
    # change `name` to reflect your computer user-name
    sys_vars = os.environ
    return ( sys_vars.get('NAME', '') == name
             or sys_vars.get('COMPUTERNAME', '') == name)

In [18]:
def return_files():
    '''mimic fastai proc for loading files, print to view order'''
    
    extensions=['.txt']

    # original directory: inode != alphabet-order
    path = Path('demo-dogs/dogs/')
    f = [o.name for o in os.scandir(path) if o.is_file()]
    res = _get_files(path, path, f, extensions)
    print(path, *res, sep='\n')

    print('-----')

    # new directory: inode == alphabet-order
    path = Path('demo-dogs/new-dogs/demo-dogs/dogs/')
    f = [o.name for o in os.scandir(path) if o.is_file()]
    res = _get_files(path, path, f, extensions)
    print(path, *res, sep='\n')

In [19]:
if not(is_local()):
    print('on GCP server')
    return_files()

on GCP server
demo-dogs/dogs
demo-dogs/dogs/dog_2.txt
demo-dogs/dogs/dog_3.txt
demo-dogs/dogs/dog_1.txt
-----
demo-dogs/new-dogs/demo-dogs/dogs
demo-dogs/new-dogs/demo-dogs/dogs/dog_2.txt
demo-dogs/new-dogs/demo-dogs/dogs/dog_3.txt
demo-dogs/new-dogs/demo-dogs/dogs/dog_1.txt


In [10]:
if is_local() and os.name == 'posix':
    print('on laptop, windows env')
    return_files()

on laptop, windows env
demo-dogs/dogs
demo-dogs/dogs/dog_1.txt
demo-dogs/dogs/dog_2.txt
demo-dogs/dogs/dog_3.txt
-----
demo-dogs/new-dogs/demo-dogs/dogs
demo-dogs/new-dogs/demo-dogs/dogs/dog_1.txt
demo-dogs/new-dogs/demo-dogs/dogs/dog_2.txt
demo-dogs/new-dogs/demo-dogs/dogs/dog_3.txt


In [6]:
if is_local() and os.name == 'nt':
    print('on laptop, windows env')
    return_files()

on laptop, windows env
demo-dogs\dogs
demo-dogs\dogs\dog_1.txt
demo-dogs\dogs\dog_2.txt
demo-dogs\dogs\dog_3.txt
-----
demo-dogs\new-dogs\demo-dogs\dogs
demo-dogs\new-dogs\demo-dogs\dogs\dog_1.txt
demo-dogs\new-dogs\demo-dogs\dogs\dog_2.txt
demo-dogs\new-dogs\demo-dogs\dogs\dog_3.txt


##### Screenshots of the inode listing for different machines

<p> local (wsl) `ls -i` </p>
<img src="misc/local_wsl_inode.PNG"/>

<p> server side `ls -i` </p>
<img src="misc/server_inode.PNG"/>